# 🌎 Plotting.

### In this Notebook we'll Do a Basic Plotting using the latest Data Points available as if they were prediction.

### Importing Necessary Libraries.

In [1]:
import json
import time
from math import sqrt, radians
import numpy as np

import pandas as pd
import geopandas as gpd

import folium
import folium.plugins

import os
from dotenv import load_dotenv

load_dotenv("../.env")

import sys

sys.path.append("../Scripts/")
sys.path.append("../")

import paths
import config

### First Things First, let's gather all our Raw Data Files in our Raw Data Directory, but Instead of creating a Gigatic DataFrame, this time we'll create one containing only the last record for each City.

In [2]:
Cities = config.Cities

os.listdir(paths.RAW_DATA_DIR)

['Perugia_HistoricalData_01092022_29022024.parquet',
 '.ipynb_checkpoints',
 'Roma_HistoricalData_01092022_29022024.parquet',
 'Bologna_HistoricalData_01092022_29022024.parquet',
 'Trento_HistoricalData_01092022_29022024.parquet',
 'Potenza_HistoricalData_01092022_29022024.parquet',
 'Firenze_HistoricalData_01092022_29022024.parquet',
 'Ancona_HistoricalData_01092022_29022024.parquet',
 'Catanzaro_HistoricalData_01092022_29022024.parquet',
 'Cagliari_HistoricalData_01092022_29022024.parquet',
 'Venezia_HistoricalData_01092022_29022024.parquet',
 'Genova_HistoricalData_01092022_29022024.parquet',
 'Milano_HistoricalData_01092022_29022024.parquet',
 'Bari_HistoricalData_01092022_29022024.parquet',
 'Campobasso_HistoricalData_01092022_29022024.parquet',
 'Trieste_HistoricalData_01092022_29022024.parquet',
 "L'Aquila_HistoricalData_01092022_29022024.parquet",
 'Palermo_HistoricalData_01092022_29022024.parquet',
 'Torino_HistoricalData_01092022_29022024.parquet',
 'Aosta_HistoricalData_0109

In [3]:
LastRecordDF = pd.DataFrame()

for file in os.listdir(paths.RAW_DATA_DIR):
    if file.endswith(".parquet"):
        #Splitting the FileName for '_' character and then getting the first item in the list as it's going to be the correspective name of the city.
        CityName = file.split("_")[0]
        print(f'Fetching Raw Data from Disk in Raw Data Dir for {CityName}')
        
        #Get the City ID for given City from Cities List
        CityID = next((x["CityID"] for x in Cities if x["CityName"] == CityName), None)
        Latitude = next((x["Latitude"] for x in Cities if x["CityName"] == CityName), None)
        Longitude = next((x["Longitude"] for x in Cities if x["CityName"] == CityName), None)

        print(f'Got City ID {CityID} with Coordinates: {Latitude, Longitude}')
        
        TempCityDF = pd.read_parquet(paths.RAW_DATA_DIR / file)
        
        #Creating a Single (Last) Record DataFrame
        
        #print([CityID, Latitude, Longitude, TempCityDF["EuropeanAQI"].iloc[-1]])
        LastRecord = pd.DataFrame(data = [[CityID, Latitude, Longitude, TempCityDF["EuropeanAQI"].iloc[-1]]], columns = ["CityID", "Latitude", "Longitude", "EuropeanAQI"])
                
        LastRecordDF = pd.concat([LastRecordDF, LastRecord])

Fetching Raw Data from Disk in Raw Data Dir for Perugia
Got City ID PG with Coordinates: (43.110718, 12.390828)
Fetching Raw Data from Disk in Raw Data Dir for Roma
Got City ID RM with Coordinates: (41.902782, 12.496365)
Fetching Raw Data from Disk in Raw Data Dir for Bologna
Got City ID BO with Coordinates: (44.494888, 11.342616)
Fetching Raw Data from Disk in Raw Data Dir for Trento
Got City ID TN with Coordinates: (46.066669, 11.12907)
Fetching Raw Data from Disk in Raw Data Dir for Potenza
Got City ID PZ with Coordinates: (40.637241, 15.80222)
Fetching Raw Data from Disk in Raw Data Dir for Firenze
Got City ID FI with Coordinates: (43.769562, 11.255814)
Fetching Raw Data from Disk in Raw Data Dir for Ancona
Got City ID AN with Coordinates: (43.615849, 13.51874)
Fetching Raw Data from Disk in Raw Data Dir for Catanzaro
Got City ID CZ with Coordinates: (38.910542, 16.587761)
Fetching Raw Data from Disk in Raw Data Dir for Cagliari
Got City ID CA with Coordinates: (39.215408, 9.10932)

In [4]:
#We Don't Care about all the Weather Data and the Datetimes, since we only need this sample.
LastRecordDF.reset_index(inplace = True)
LastRecordDF.drop("index", axis = 1, inplace = True)
LastRecordDF

,CityID,Latitude,Longitude,EuropeanAQI
0,PG,43.110718,12.390828,22
1,RM,41.902782,12.496365,27
2,BO,44.494888,11.342616,38
3,TN,46.066669,11.129070,63
4,PZ,40.637241,15.802220,17
5,FI,43.769562,11.255814,24
6,AN,43.615849,13.518740,34
7,CZ,38.910542,16.587761,26
8,CA,39.215408,9.109320,17
9,VE,45.440845,12.315515,33


### Next Up, let's create a GeoDataFrame for our LastRecordDF and add the geometry column with the Coordinates

In [5]:
Geometry = gpd.points_from_xy(LastRecordDF["Latitude"], LastRecordDF["Longitude"])
GeoDF = gpd.GeoDataFrame(
    LastRecordDF, geometry = Geometry
)

GeoDF

,CityID,Latitude,Longitude,EuropeanAQI,geometry
0,PG,43.110718,12.390828,22,POINT (43.11072 12.39083)
1,RM,41.902782,12.496365,27,POINT (41.90278 12.49637)
2,BO,44.494888,11.342616,38,POINT (44.49489 11.34262)
3,TN,46.066669,11.129070,63,POINT (46.06667 11.12907)
4,PZ,40.637241,15.802220,17,POINT (40.63724 15.80222)
5,FI,43.769562,11.255814,24,POINT (43.76956 11.25581)
6,AN,43.615849,13.518740,34,POINT (43.61585 13.51874)
7,CZ,38.910542,16.587761,26,POINT (38.91054 16.58776)
8,CA,39.215408,9.109320,17,POINT (39.21541 9.10932)
9,VE,45.440845,12.315515,33,POINT (45.44085 12.31551)


### Then let's Create our Map.

In [6]:
Map = folium.Map(location=[41.50, 16.50], tiles = "Esri.WorldPhysical", zoom_start = 6,
               zoom_control = True,
               scrollWheelZoom = True,
               dragging = True)

In [7]:
Map

### Now, let's add Markers for each City to the Map, with Respective City Name

In [8]:
for row in LastRecordDF.iterrows():
    
    CityID = row[1]["CityID"]
    Latitude = row[1]["Latitude"]
    Longitude = row[1]["Longitude"]
    EuropeanAQI = row[1]["EuropeanAQI"]

    if EuropeanAQI < 20:
        Color = "green"
    
    elif EuropeanAQI < 40 and EuropeanAQI >= 20:
        Color = "beige"
    
    elif EuropeanAQI < 60 and EuropeanAQI >= 40:
        Color = "orange"
    
    elif EuropeanAQI < 80 and EuropeanAQI >= 60:
        Color = "red"
    
    elif EuropeanAQI < 100 and EuropeanAQI >= 80:
        Color = "darkred"
    
    elif EuropeanAQI >= 100:
        color = "black"

    #Place the markers with the popup labels and data
    Map.add_child(folium.Marker(
                                location = (Latitude, Longitude),
                                popup =
                                "CityID: " + str(CityID) + "<br>" + "PredictedAQI: " + str(EuropeanAQI) + "<br>",
                                icon = folium.Icon(color="%s" % Color),
                            )
                        )

In [9]:
Map

### Lastly, we need to Create our Heat Data to give those HeatMap vibes and to give context to our map visualization.

In [10]:
HeatData = []

for row in LastRecordDF.iterrows():
    CityID = row[1]["CityID"]
    Coordinates = [row[1]["Latitude"], row[1]["Longitude"]]
    EuropeanAQI = row[1]["EuropeanAQI"]
    Data = [row[1]["Latitude"], row[1]["Longitude"], EuropeanAQI]
    
    #The Heatmap wants Data in format [Latitude, Longitude] or [Latitude, Longitude, Value]
    HeatData.append(Data)
    

HeatMap = Map
folium.plugins.HeatMap(HeatData, radius = 50, blur = 50).add_to(HeatMap)
HeatMap

### As you can see, our Map is quite Explainatory.
### In Northern Italy Air Quality is worse due to Industrial Activity, so our Data makes sense.
### Also, Northern Cities with that usually are windy, such as Genova and Trieste, have lower AQI.
### However, that being said, our Map is still not as good as I would like it to be, so let's tweak it a bit.
### First, we would need to add coordinates points for each ray going from the Each City to the nearest one.
### Then we would need to make AQI as if it's transitioning from one city to another, to give that "blur" effect and not leave empty spaces.
### Lastly we need to tweak the color palette, so we make sure that each city has a more appropriate color.

In [11]:
#Let's First create a copy of our previous Map.
NewHeatMap = Map

In [12]:
#Let's Copy our LastRecordDF and Sort it by Latitude.
NewLastRecordDF = LastRecordDF.copy()
NewLastRecordDF.sort_values(by = "Latitude", inplace = True, ascending = False)

for row in NewLastRecordDF.iterrows():
    print(row[1])

CityID                TN
Latitude       46.066669
Longitude       11.12907
EuropeanAQI           63
Name: 3, dtype: object
CityID                AO
Latitude       45.735062
Longitude        7.31308
EuropeanAQI           26
Name: 18, dtype: object
CityID                TS
Latitude       45.653599
Longitude       13.77852
EuropeanAQI           33
Name: 14, dtype: object
CityID                MI
Latitude       45.464203
Longitude       9.189982
EuropeanAQI           73
Name: 11, dtype: object
CityID                VE
Latitude       45.440845
Longitude      12.315515
EuropeanAQI           33
Name: 9, dtype: object
CityID                TO
Latitude       45.070339
Longitude       7.686864
EuropeanAQI           66
Name: 17, dtype: object
CityID                BO
Latitude       44.494888
Longitude      11.342616
EuropeanAQI           38
Name: 2, dtype: object
CityID                GE
Latitude       44.405651
Longitude       8.946256
EuropeanAQI           27
Name: 10, dtype: object
CityID     

In [13]:
#Now that our DataFrame is sorted in Descending order, let's fill it with coordinates of Intermidiate Points in the Map.

#Just to give an Example, say that we want 10 Intermediate Points between Point A with Coordinates (5, 1) and Point B with Coordinates (3, 2)

latitudes = np.linspace(5, 3, 10 + 2)
longitudes = np.linspace(1, 2, 10 + 2)
intermediates = [(lat, lon) for lat, lon in zip(latitudes, longitudes)]
print(intermediates)

#Now, how do we define how many intermediate Points we want for each Pair A-B?
#As a rule of thumb, we'll add a point for every 0.1 fifference (rounded for excess) in Latitude and 1 point for every 0.10 difference in Longitude
#And how do we find for which Pair of Cities (Point A and B) we would like to have those intermediate Points?
#As another rule of thumb, we'll DEFINE MANUALLY all the pairs of points we would like to get the Intermediate Points.
#Going Manual on this task might seem time consuming, but as the DataFrame has only 20 entries, it's no drama, a better approach for bigger dataframes would be to find X nearest neighbors.
#We can do so much more Engineering on this Map, but I think this will do the work as we want a very simple viz.

[(5.0, 1.0), (4.818181818181818, 1.0909090909090908), (4.636363636363637, 1.1818181818181819), (4.454545454545455, 1.2727272727272727), (4.2727272727272725, 1.3636363636363638), (4.090909090909091, 1.4545454545454546), (3.909090909090909, 1.5454545454545454), (3.7272727272727275, 1.6363636363636362), (3.5454545454545454, 1.7272727272727273), (3.3636363636363633, 1.8181818181818183), (3.1818181818181817, 1.9090909090909092), (3.0, 2.0)]


In [14]:
PairsIDsToDo = [["AO", "TO"],
                ["AO", "TN"],
                ["AO", "MI"],
                ["TO", "MI"],
                ["TO", "GE"],
                ["MI", "GE"],
                ["MI", "TN"],
                ["MI", "BO"],
                ["MI", "VE"],
                ["GE", "BO"],
                ["GE", "FI"],
                ["TN", "VE"],
                ["TN", "TS"],
                ["VE", "TS"],
                ["VE", "BO"],
                ["BO", "AN"],
                ["BO", "FI"],
                ["FI", "RM"],
                ["FI", "PG"],
                ["FI", "AN"],
                ["PG", "AN"],
                ["PG", "AQ"],
                ["PG", "RM"],
                ["AN", "AQ"],
                ["RM", "AQ"],
                ["RM", "NA"],
                ["RM", "CB"],
                ["AQ", "CB"],
                ["NA", "CB"],
                ["NA", "PZ"],
                ["CB", "BA"],
                ["CB", "PZ"],
                ["PZ", "BA"],
                ["PZ", "CZ"],
                ["CZ", "PA"],
                #["PA", "CA"]
               ]

#It took less than 5 Minutes, let's now plot a line connecting all points to see if they are correct.

In [15]:
for pair in PairsIDsToDo:

    PointA = NewLastRecordDF[NewLastRecordDF["CityID"] == pair[0]]
    PointB = NewLastRecordDF[NewLastRecordDF["CityID"] == pair[1]]
    
    PointA_Coords = [PointA["Latitude"].values[0], PointA["Longitude"].values[0]]
    PointB_Coords = [PointB["Latitude"].values[0], PointB["Longitude"].values[0]]
    
    folium.PolyLine(locations=[PointA_Coords, PointB_Coords], color='blue').add_to(NewHeatMap)

In [16]:
NewHeatMap

### Perfect, now, let's Move on with creating the intermediate points and let's Reecreate our Map and place the Markers.

In [17]:
#Copying the DF
FilledLastRecordDF = NewLastRecordDF.copy()

#Defining a Function to calculate the Euclidean Distance as if Earth was Flat.
#I DO NOT BELIEVE EARTH IS FLAT, but in shorter distances and far from the poles, earth curvature is negotiable.
#A more precise method would be calculating Haversine Distance.
def EuclideanDistance(Point1_Coords, Point2_Coords):
    #Convert latitude and longitude from degrees to radians
    Lat1, Lon1, Lat2, Lon2 = map(radians, [Point1_Coords[0], Point1_Coords[1], Point2_Coords[0], Point2_Coords[1]])

    #Calculate differences in latitude and longitude
    LatDiff = abs(Lat2 - Lat1)
    LonDiff = abs(Lon2 - Lon1)
    
    #Calculate Euclidean distance
    Distance = sqrt(LatDiff**2 + LonDiff**2)
    #print(f'Distance: {round(Distance, 6)}')
    
    return round(Distance, 6)

#Defining a Function to Calculate the Synthetic EuropeanAQI of Intermediate Point.
def CalculateSyntheticEAQI(EAQI_A, EAQI_B, PointA_Coords, PointB_Coords, PointX_Coords):
    
    EAQI_Diff = EAQI_B - EAQI_A
    DistancePercentage = EuclideanDistance(PointA_Coords, PointX_Coords) / EuclideanDistance(PointA_Coords, PointB_Coords)
    EAQI_X = EAQI_A + EAQI_Diff * DistancePercentage
    
    #print(f'EAQI_A: {EAQI_A}')
    #print(f'EAQI_X: {int(EAQI_X)}')
    #print(f'EAQI_B: {EAQI_B}')
    
    return int(EAQI_X)

for pair in PairsIDsToDo:

    #Getting PointA and PointB rows given 2 City IDs, then getting the coordinates, generating the intermediate point and lastly, inserting a "fake" City ID as First Element
    #and a Synthetic European AQI as Last Element.
    PointA = NewLastRecordDF[NewLastRecordDF["CityID"] == pair[0]]
    PointB = NewLastRecordDF[NewLastRecordDF["CityID"] == pair[1]]
    
    PointA_Coords = [PointA["Latitude"].values[0], PointA["Longitude"].values[0]]
    PointB_Coords = [PointB["Latitude"].values[0], PointB["Longitude"].values[0]]
    
    NumIntermediates = int(abs(PointA_Coords[0] - PointB_Coords[0]) * 10) + int(abs(PointA_Coords[1] - PointB_Coords[1]) * 10)
    
    Lats = np.linspace(PointA_Coords[0], PointB_Coords[0], NumIntermediates + 2)
    Longs = np.linspace(PointA_Coords[1], PointB_Coords[1], NumIntermediates + 2)
    
    IntermediatePoints = [[round(lat, 6), round(lon, 6)] for lat, lon in zip(Lats, Longs)]
    #Cutting off First and Last Elements as they are PointA and PointB
    IntermediatePoints = IntermediatePoints[1:-1]
    
    print(f'Start Point: {PointA_Coords}')
    print(f'Intermediates: {IntermediatePoints}')
    print(f'End Point: {PointB_Coords}')
    
    for intermediate in IntermediatePoints:
        IntermediateCoords = intermediate.copy()
        
        intermediate.insert(0, "IntermediatePoint")
        #Let's add those points to the DataFrame with a "IntermediatePoint" ID and a EuropeanAQI Value
        intermediate.append(CalculateSyntheticEAQI(PointA["EuropeanAQI"].values[0], PointB["EuropeanAQI"].values[0], PointA_Coords, PointB_Coords, IntermediateCoords))
    
        FilledLastRecordDF = pd.concat([FilledLastRecordDF, pd.DataFrame([[x for x in intermediate]], columns = FilledLastRecordDF.columns)], ignore_index=True)

Start Point: [45.735062, 7.31308]
Intermediates: [[45.66859, 7.350458], [45.602117, 7.387837], [45.535645, 7.425215], [45.469173, 7.462594], [45.4027, 7.499972], [45.336228, 7.53735], [45.269756, 7.574729], [45.203284, 7.612107], [45.136811, 7.649486]]
End Point: [45.070339, 7.686864]
Start Point: [45.735062, 7.31308]
Intermediates: [[45.742957, 7.403937], [45.750853, 7.494794], [45.758748, 7.585651], [45.766644, 7.676508], [45.774539, 7.767365], [45.782434, 7.858221], [45.79033, 7.949078], [45.798225, 8.039935], [45.806121, 8.130792], [45.814016, 8.221649], [45.821911, 8.312506], [45.829807, 8.403363], [45.837702, 8.49422], [45.845598, 8.585077], [45.853493, 8.675934], [45.861388, 8.76679], [45.869284, 8.857647], [45.877179, 8.948504], [45.885075, 9.039361], [45.89297, 9.130218], [45.900865, 9.221075], [45.908761, 9.311932], [45.916656, 9.402789], [45.924552, 9.493646], [45.932447, 9.584503], [45.940343, 9.67536], [45.948238, 9.766216], [45.956133, 9.857073], [45.964029, 9.94793], [45

In [18]:
#Let's sort again this last DataFrame by Latitude.
SortedFilledLastRecordDF = FilledLastRecordDF.copy()
SortedFilledLastRecordDF.sort_values(by = "Latitude", inplace = True, ascending = False)
SortedFilledLastRecordDF

,CityID,Latitude,Longitude,EuropeanAQI
0,TN,46.066669,11.129070,63
69,IntermediatePoint,46.058774,11.038213,62
293,IntermediatePoint,46.053344,11.214536,62
68,IntermediatePoint,46.050878,10.947356,61
162,IntermediatePoint,46.043497,11.054490,63
...,...,...,...,...
751,IntermediatePoint,38.195150,13.684099,22
752,IntermediatePoint,38.175278,13.603442,22
753,IntermediatePoint,38.155406,13.522785,22
754,IntermediatePoint,38.135534,13.442127,22


### Now, let's recreate the map, place the Markers (Colored for Actual Data and Gray for Synthetic) and draw the lines to see if they fit well.

In [19]:
#Creating the Map
FinalMap = folium.Map(location=[41.50, 16.50], tiles = "Esri.WorldPhysical", zoom_start = 6,
               zoom_control = True,
               scrollWheelZoom = True,
               dragging = True)

#Adding Markers, Colored for Real Data, Gray for Synthetic One.
for row in SortedFilledLastRecordDF.iterrows():
    
    CityID = row[1]["CityID"]
    Latitude = row[1]["Latitude"]
    Longitude = row[1]["Longitude"]
    EuropeanAQI = row[1]["EuropeanAQI"]

    if CityID != "IntermediatePoint": 
        
        if EuropeanAQI < 20:
            Color = "green"

        elif EuropeanAQI < 40 and EuropeanAQI >= 20:
            Color = "beige"

        elif EuropeanAQI < 60 and EuropeanAQI >= 40:
            Color = "orange"

        elif EuropeanAQI < 80 and EuropeanAQI >= 60:
            Color = "red"

        elif EuropeanAQI < 100 and EuropeanAQI >= 80:
            Color = "darkred"

        elif EuropeanAQI >= 100:
            Color = "black"

    else:
        Color = "gray"
        
    #Place the markers with the popup labels and data
    FinalMap.add_child(folium.Marker(
                                location = (Latitude, Longitude),
                                popup =
                                "CityID: " + str(CityID) + "<br>" + "PredictedAQI: " + str(EuropeanAQI) + "<br>",
                                icon = folium.Icon(color="%s" % Color),
                            )
                        )

#Adding Lines that connect Points.
for pair in PairsIDsToDo:

    PointA = NewLastRecordDF[NewLastRecordDF["CityID"] == pair[0]]
    PointB = NewLastRecordDF[NewLastRecordDF["CityID"] == pair[1]]
    
    PointA_Coords = [PointA["Latitude"].values[0], PointA["Longitude"].values[0]]
    PointB_Coords = [PointB["Latitude"].values[0], PointB["Longitude"].values[0]]
    
    folium.PolyLine(locations=[PointA_Coords, PointB_Coords], color='blue').add_to(FinalMap)

In [20]:
#Render Map
FinalMap

#P.S.: It seems messy at first, but just Zoom in and you'll know it's perfect.

### Finally, one last thing: let's reecreate the Map one final time, but with the updated heatmap.

In [21]:
#Creating the Map
DefinitiveMap = folium.Map(location=[41.50, 16.50], tiles = "Esri.WorldPhysical", zoom_start = 6,
               zoom_control = False,
               scrollWheelZoom = False,
               dragging = False)

#Adding Markers, Only Colored for Real Data Ones.
for row in SortedFilledLastRecordDF.iterrows():
    
    CityID = row[1]["CityID"]
    Latitude = row[1]["Latitude"]
    Longitude = row[1]["Longitude"]
    EuropeanAQI = row[1]["EuropeanAQI"]
    
    if CityID != "IntermediatePoint":
    
        if EuropeanAQI < 20:
            Color = "green"

        elif EuropeanAQI < 40 and EuropeanAQI >= 20:
            Color = "beige"

        elif EuropeanAQI < 60 and EuropeanAQI >= 40:
            Color = "orange"

        elif EuropeanAQI < 80 and EuropeanAQI >= 60:
            Color = "red"

        elif EuropeanAQI < 100 and EuropeanAQI >= 80:
            Color = "darkred"

        elif EuropeanAQI >= 100:
            Color = "black"

        #Place the markers with the popup labels and data
        DefinitiveMap.add_child(folium.Marker(
                                    location = (Latitude, Longitude),
                                    popup =
                                    "CityID: " + str(CityID) + "<br>" + "PredictedAQI: " + str(EuropeanAQI) + "<br>",
                                    icon = folium.Icon(color="%s" % Color),
                                )
                            )

#Not Adding Lines that connect Points, but generating the Heatmap.
HeatData = []

for row in SortedFilledLastRecordDF.iterrows():
    CityID = row[1]["CityID"]
    Coordinates = [row[1]["Latitude"], row[1]["Longitude"]]
    EuropeanAQI = row[1]["EuropeanAQI"]
    Data = [row[1]["Latitude"], row[1]["Longitude"], EuropeanAQI]
    
    #The Heatmap wants Data in format [Latitude, Longitude] or [Latitude, Longitude, Value]
    HeatData.append(Data)
    
folium.plugins.HeatMap(HeatData, radius = 40, blur = 55, min_opacity = 0.55, gradient = {0.4: "lime", 0.77: "yellow", 1: "red"}).add_to(DefinitiveMap)

In [22]:
DefinitiveMap

### TA-DAAAA!!! 
### It's not the best map ever existed, but does its job.
### We can do a ton of engineering when plotting, we can get more data for smaller cities, we can use a different framework or we can fill more and more synthetic data to make it as dense as possible.
### For what's worth, I'm happy as it is now, so I won't be wasting too much time on this and I'll move on.